# pip install

In [ ]:
pip install gymnasium


In [ ]:
pip install "stable-baselines3[extra]>=2.0.0a4"

In [ ]:
import numpy as np

import gymnasium as gym
from gymnasium import spaces

from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env, SubprocVecEnv, DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

## set reproducibility

In [ ]:
import torch
import numpy as np
import random
import os
os.environ['PYTHONASHSEED'] = '0'
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

seed = 0
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

# Learning Rate Func

In [ ]:

def linear_schedule(initial_value):

    if isinstance(initial_value, str):
        initial_value = float(initial_value)

    def func(progress):

        return progress * initial_value

    return func

# Ranking Func

In [ ]:
    def load_topic_target_location(topic_id,target_recall):
      ## load data

      vector_size = 100 # vector size to feed NN

      all_vectors = [[-1]*vector_size for i in range(vector_size)]

      target_location = -1 # initial


      n_docs = len(doc_rank_dic[topic_id])  # total n. docs in topic
      rel_list = rank_rel_dic[topic_id]  # list binary rel of ranked docs

      # get points
      windows = make_windows(vector_size, n_docs)

      window_size = windows[0][1]

      # calculate points that will be used to fit curve
      rel_cnt,rel_rate, n_samp_docs_current = get_rel_cnt_rate(windows, vector_size, rel_list)


      n_rel = sum(rel_cnt)
      prev = sum(rel_cnt)/n_docs


      #update all vector with all possible examined states
      for i in range(vector_size):
        all_vectors[i][0:i+1] = rel_rate[0:i+1] # update examined part

        #calculate target recall stopping pos
        #mark only 1st recall achieved stopping position
        if (sum(rel_cnt[0:i+1]) / sum(rel_cnt)) >= target_recall and target_location == -1:
          target_location = i


      return topic_id, n_docs, n_rel, prev, target_location

In [ ]:
def get_rel_cnt_rate(windows, window_size, rel_list):

    # x-values are the cnt at which relevant documents occur in the window
    x = [np.sum(rel_list[w_s:w_e]) for (w_s,w_e) in windows]

    # y-values are the rate at which relevant documents occur in the window
    y = [np.sum(rel_list[w_s:w_e]) for (w_s,w_e) in windows]
    y = [y_i/window_size for y_i in y]


    # z-values are the cnt of documents in the window
    z = [len(rel_list[w_s:w_e]) for (w_s,w_e) in windows]


    # convert lists to numpy arrays
    x = np.array(x)
    y = np.array(y)
    z= np.array(z)
    return (x,y,z)

In [ ]:
%matplotlib inline

# IMPORT LIBRARIES
import sys
import os
import numpy as np
import pandas as pd
import math
from scipy.optimize import curve_fit
import random
import glob
import subprocess
import matplotlib.pyplot as plt
from scipy.integrate import simps
from scipy.stats import norm
import os

import scipy

import seaborn as sns


In [ ]:
# Ensure that Python looks in correct place for local modules
DIR = '/xxhome/' # replace with utils home directory


sys.path.append(DIR)

# IMPORT OWN FUNCTIONS
from utils.read_data_fns import *
from utils.eval_fns import *
from utils.inhomogeneous_pp_fns import *

In [ ]:
# LOAD TOPIC RELEVANCE DATA
def load_rel_data(qrels):
  qrel_fname =  os.path.join(DIR, qrels)
  with open(qrel_fname, 'r') as infile:
      qrels_data = infile.readlines()
  query_rel_dic = make_rel_dic(qrels_data) # make dictionary of list of docids relevant to each queryid

  return qrel_fname, query_rel_dic

In [ ]:
# LOAD RUN DATA
def load_run_data(run):
  run_fname = os.path.join(DIR, run)
  with open(run_fname, 'r') as infile:
    run_data = infile.readlines()
  doc_rank_dic = make_rank_dic(run_data)  # make dictionary of ranked docids for each queryid
  rank_rel_dic = make_rank_rel_dic(query_rel_dic,doc_rank_dic) # make dic of list relevances of ranked docs for each queryid

  return doc_rank_dic, rank_rel_dic

In [ ]:
df_all_targets = pd.DataFrame()
DRL_DIR = '/xx/' # replace with working directory


# TAREnv class



In [ ]:
import numpy as np


import gymnasium as gym
from gymnasium import spaces

SELECTED_TOPICS = [] # keep track of all randomly selected topics

class TAREnv(gym.Env):



    def __init__(self, target_recall, topics_list = None, topic_id= None, size=100 , render_mode=None):
        self.size = size  # The size of the ranking relv vector

        #observation is 1D np array size array of relv vector
        self.observation_space = spaces.Box(-1,  1, shape=(size,), dtype=np.float32)

        #  actions, corresponding to "next", "stop"
        self.action_space = spaces.Discrete(2)

        # Set up properties
        self.done = False
        self.reward = 0
        self.hit, self.miss = 0, 0


        # Set up the TAR
        self.vector_size = size


        # current position and stop position
        self._agent_location = 0
        self._target_location = -1 #dummy value

        # keep predicted recall so far
        self.recall = 0
        self.target_recall = target_recall

        # topic data
        self.topics_list = topics_list
        self.topic_id = topic_id # for single env


        #for vec env
        if topic_id is None:
          # select a new random topic
          while True:
            t = random.choice(topics_list)
            if t not in SELECTED_TOPICS:
              SELECTED_TOPICS.append(t)
              self.topic_id = t
              break

          # use same ordered list of topics across diffreent runs
          if TRAINING:
            global SELECTED_TOPICS_ORDERERD_INDEX
            self.topic_id = SELECTED_TOPICS_ORDERERD[SELECTED_TOPICS_ORDERERD_INDEX]
            SELECTED_TOPICS_ORDERERD_INDEX += 1
        else:
           self.topic_id = topic_id # for single env


        self.n_docs = 0
        self.rel_cnt = []
        self.rel_rate = []
        self.n_samp_docs = 0
        self.n_samp_docs_after_target = 0
        self.n_samp_docs_current = []
        self.rel_list = []
        self.all_vectors = []

        # Define constants for clear code
        self.NEXT = 0
        self.STOP = 1


        self.load_data_flag = True
        self.load_data(self.topic_id)

        self.first_step_flag = True




    def load_data(self, topic_id):

      all_vectors = [[-1]*self.vector_size for i in range(self.vector_size)]
      topic_id = self.topic_id

      n_docs = len(doc_rank_dic[topic_id])  # total n. docs in topic
      rel_list = rank_rel_dic[topic_id]  # list binary rel of ranked docs

      # get points
      windows = make_windows(self.vector_size, n_docs)

      window_size = windows[0][1]

      # calculate relv vector points (batches)
      rel_cnt,rel_rate, n_samp_docs_current = get_rel_cnt_rate(windows, self.vector_size, rel_list)

      self.n_docs = n_docs
      self.rel_cnt = rel_cnt
      self.rel_rate = rel_rate
      self.n_samp_docs_current = n_samp_docs_current
      self.rel_list = rel_list


      #update all vector with all possible examined states
      for i in range(self.vector_size):
        all_vectors[i][0:i+1] = rel_rate[0:i+1] # update examined part

        #calculate target recall stopping pos
        #mark only 1st recall achieved stopping position
        if (sum(self.rel_cnt[0:i+1]) / sum(self.rel_cnt)) >= self.target_recall and self._target_location == -1:
          self._target_location = i


      self.all_vectors = all_vectors


    def _get_obs(self):
        return  np.array(self.all_vectors[self._agent_location], dtype=np.float32)


    def _get_info(self):
        return {
                "topic_id": self.topic_id,
                "recall": round((self.recall),3),
                "cost": round(((self._agent_location +1)/100),3), # each vec pos == 1% of collection
                "e_cost": round((((self._agent_location +1) - self._target_location)/100) / (1-(self._target_location/100)),3), #updated
                "distance": (self._agent_location - self._target_location),
                "agent": (self._agent_location),
                "target": (self._target_location),
                "agent_vector": np.array(self.all_vectors[self._agent_location]),
                "terminal_observation": np.array(self.all_vectors[self._target_location])} # target_vector named terminal_observation needed for SB3 vec_env

    def reset(self,seed=0):

        # re-load data 1st time for vec_env
        if self.load_data_flag:
          self.load_data(self.topic_id)
          self.load_data_flag = False

        #initialize all pos info in reset (i.e. recall, n_samp_docs)
        # always start at first position of relv vector (1st batch size)
        self._agent_location = 0
        self.n_samp_docs =  sum(self.n_samp_docs_current[0:self._agent_location+1])
        self.n_samp_docs_after_target =  sum(self.n_samp_docs_current[self._target_location:self._agent_location+1])

        self.recall = sum(self.rel_cnt[0:self._agent_location+1]) / sum(self.rel_cnt)

        state = self.all_vectors[self._agent_location]


        observation = self._get_obs()
        info = self._get_info()

        return observation, info


    def step(self, action):
        truncated = False
        terminated = False

        if self._agent_location >= self.vector_size-1:
          self.done = True
          truncated = True


        if self._agent_location >= self.vector_size-2 and action == self.NEXT:
          self.done = True
          truncated = True

        if action == self.STOP:
            terminated = True


        if action == self.NEXT:
            #update TAR vars to next pos vars
            if self.first_step_flag:
              self._agent_location = self._agent_location # dont move next, examine 1st portion at pos [0]
              self.first_step_flag = False
            else:
              self._agent_location += 1 # move to next portion (examined)

            self.n_samp_docs =  sum(self.n_samp_docs_current[0:self._agent_location+1])
            self.n_samp_docs_after_target =  sum(self.n_samp_docs_current[self._target_location:self._agent_location+1])
            self.recall = sum(self.rel_cnt[0:self._agent_location+1]) / sum(self.rel_cnt)

        observation = self._get_obs()
        info = self._get_info()


        # R(t) = 1 - (t-1) / target_step, if t ≤ target_step , -1 * (t - target_step) / (total_steps - target_step), if t > target_step

        #to get more easy readable formula
        reward_target_location = self._target_location+1
        reward_agent_location = self._agent_location+1

        # misses/reach target_recall
        if reward_agent_location <= reward_target_location:
          self.reward = 1 - ((reward_agent_location-1) / reward_target_location)

        # overachieves target_recall
        elif reward_agent_location > reward_target_location:
            self.reward = -1 * (reward_agent_location - reward_target_location) / (self.vector_size - reward_target_location)

        return observation, self.reward, terminated, truncated, info




    def render(self):
        # we dont need render
        return


    def close(self):
        # we dont need close
        return



## Hyperparameter Settings

In [ ]:
TRAINING = True
total_runs = 10

# Train the agent
ent_coef = 0.01

gamma = 0.99
learning_rate_initial = 0.0001

learning_rate = linear_schedule(learning_rate_initial)
clip_range=0.2

n_steps = 100
batch_size = 100
n_epochs =8

model_name = 'reward_1-1_'
learning_rate_type = '_linear_schedule'+str(learning_rate_initial)

learning_rate_type = '_lr_static'+str(learning_rate_initial)

total_timesteps = 100_000

tensorboard_log = '/logs/'



# Target 0.9

In [ ]:
target_recall = 0.9 # replace with other target recall level

## Training

In [ ]:
TRAINING = True

In [ ]:
training_dataset = 'CLEF'


#### sort topics by target location

In [ ]:

dataset_name = 'CLEF2017'


qrels = "data/qrels/CLEF2017_qrels.txt"


qrel_fname, query_rel_dic = load_rel_data(qrels)
print("Number of topics:", len(query_rel_dic))

run = "data/rankings/clef2017_training_ranking.txt"

doc_rank_dic, rank_rel_dic = load_run_data(run)

topics_list = make_topics_list(doc_rank_dic,1)

In [ ]:
#remove topic CD008760 last element, contains 64 items only, < 100 vector size
topics_list= topics_list[:-1]


In [ ]:
topics_info = []

for t in topics_list:
  topic_id, n_docs, n_rel, prev, target_location = load_topic_target_location(t,target_recall)
  print(topic_id, n_docs, n_rel, round(prev,3), target_location)
  topics_info.append([topic_id, n_docs, n_rel, prev, target_location])

topics_info

In [ ]:
# import pandas as pd
import pandas as pd


df = pd.DataFrame(topics_info, columns=['topic_id', 'n_docs', 'n_rel', 'prev', 'target_location'])
df = df.sort_values(by=['target_location'])
df

In [ ]:
sorted_target_loc_topics = list(df['topic_id'])
sorted_target_loc_topics

####ordered topics

In [ ]:
TRAINING = True

SELECTED_TOPICS_ORDERERD = sorted_target_loc_topics
SELECTED_TOPICS_ORDERERD_INDEX = 0

# Instantiate the vec env

#random topic selection for each env instance
SELECTED_TOPICS = [] # reset before/after each call, keep track of all randomly selected topics

vec_env = make_vec_env(TAREnv, n_envs=len(topics_list), env_kwargs=dict(target_recall=target_recall, topics_list = topics_list, topic_id=None, size=100, render_mode='human'))

SELECTED_TOPICS = [] # reset before/after each call, keep track of all randomly selected topics

train_size = len(topics_list)
vec_env_train = vec_env

#### PPO

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:

tb_log_name = model_name+"_"+training_dataset+"_ppo_gma_"+str(gamma)+"_nsteps"+str(n_steps)+"_btch"+str(batch_size)+"_timesteps_"+str(total_timesteps)+ "_ent_coef"+str(ent_coef)+ learning_rate_type +"_n_epochs"+str(n_epochs)+"_target"+str(target_recall)

model = PPO(
    policy = 'MlpPolicy',
    env = vec_env_train,
    n_steps = n_steps,
    batch_size = batch_size,
    n_epochs = n_epochs,
    gamma = gamma,
    gae_lambda = 0.98,
    ent_coef = ent_coef,
    verbose=1,
    learning_rate = learning_rate,
    seed=0,
    tensorboard_log= tensorboard_log)

model.learn(total_timesteps=total_timesteps, tb_log_name=tb_log_name)


model.save(tensorboard_log+'model_'+tb_log_name)




In [ ]:
%tensorboard --logdir "$tensorboard_log"


## TESTING

In [ ]:
TRAINING = False

###clef2017

In [ ]:

dataset_name = 'CLEF2017'


qrels = "data/qrels/CLEF2017_qrels.txt"


qrel_fname, query_rel_dic = load_rel_data(qrels)
print("Number of topics:", len(query_rel_dic))

run = "data/rankings/clef2017_ranking.txt"

doc_rank_dic, rank_rel_dic = load_run_data(run)

topics_list = make_topics_list(doc_rank_dic,1)  # sort topics by no docs

In [ ]:
# Instantiate the vec env

#random topic selection for each env instance
SELECTED_TOPICS = [] # reset before/after each call, keep track of all randomly selected topics

vec_env = make_vec_env(TAREnv, n_envs=len(topics_list), env_kwargs=dict(target_recall=target_recall, topics_list = topics_list, topic_id=None, size=100, render_mode='human'))

SELECTED_TOPICS = [] # reset before/after each call, keep track of all randomly selected topics


In [ ]:
test_size = len(topics_list)
vec_env_test = vec_env

#### TAR Eval

In [ ]:

df = pd.DataFrame()
df_all_runs = pd.DataFrame()

for run in range(total_runs):

  # Test the trained agent
  vec_env_test = vec_env
  obs = vec_env_test.reset()
  test_steps = 100


  n_env = test_size
  agent=0
  target=0
  agent_vector=[]
  terminal_observation=[]

  topics = []
  recalls = []
  costs=[]
  e_costs = []
  reliabilities = []
  rewards = []
  distances = []
  differences = []
  targets = []
  run_cnts = []

  for eID in range(test_size):

    env = vec_env_test.envs[eID]
    obs, info = env.reset()

    for step in range(test_steps):
      action, _ = model.predict(obs, deterministic=False) # predict all next steps
      obs, reward, done, trun,info = env.step(action)


      if done or trun:
                  topic_id = info['topic_id']
                  recall = info['recall']
                  cost = info['cost']
                  e_cost =  ((info['agent'] - info['target']) / (100-info['target']))
                  distance = info['distance']

                  agent = info['agent']
                  target = info['target']
                  agent_vector = info['agent_vector']
                  terminal_observation = info['terminal_observation']

                  difference = target_recall - recall

                  reliability = 1 if recall >= target_recall else 0
                  topics.append(topic_id)
                  recalls.append(recall)
                  costs.append(cost)
                  e_costs.append(e_cost)
                  reliabilities.append(reliability)
                  rewards.append(reward)
                  distances.append(distance)
                  targets.append(target)
                  run_cnts.append(run)
                  differences.append(difference)

                  df_tmp = pd.DataFrame( list(zip([dataset_name]*len(topics_list), topics, run_cnts, recalls, reliabilities, costs, e_costs, rewards, differences, distances, targets)),
                  columns =['Dataset', 'Topic', 'Run', 'Recall', 'Reliability', 'Cost', 'e-Cost', 'Reward', 'Difference', 'Distance', 'Target'])

                  df = pd.concat([df_tmp])

                  df.groupby('Topic').mean()

                  break

  display(df)
  df.groupby('Topic').mean()
  df_all_runs = pd.concat([df_all_runs, df])

df_all_runs['Model'] = model_name
df_all_runs['Model_settings'] = tb_log_name
df_all_runs['Target_Recall'] = target_recall
df_all_runs

In [ ]:
display(df_all_runs.groupby('Topic').mean())
display(df_all_runs.groupby('Topic').std())


#### df_all_targets

In [ ]:

df_all_targets = pd.concat([df_all_targets, df_all_runs], ignore_index = True)


display(df_all_targets)

df_all_targets.describe()

In [ ]:
display(df_all_targets.groupby(['Target_Recall','Dataset']).mean().round(3))
display(df_all_targets.groupby(['Target_Recall','Dataset']).std().round(3))

###clef2018

In [ ]:

dataset_name = 'CLEF2018'


qrels = "data/qrels/CLEF2018_qrels_LiKs.txt" # use the same qrel list as their rankings


qrel_fname, query_rel_dic = load_rel_data(qrels)
print("Number of topics:", len(query_rel_dic))

run = "data/rankings/clef2018_ranking.txt"

doc_rank_dic, rank_rel_dic = load_run_data(run)

topics_list = make_topics_list(doc_rank_dic,1)  # sort topics by no docs

In [ ]:
# Instantiate the vec env

#random topic selection for each env instance
SELECTED_TOPICS = [] # reset before/after each call, keep track of all randomly selected topics

vec_env = make_vec_env(TAREnv, n_envs=len(topics_list), env_kwargs=dict(target_recall=target_recall, topics_list = topics_list, topic_id=None, size=100, render_mode='human'))

SELECTED_TOPICS = [] # reset before/after each call, keep track of all randomly selected topics


In [ ]:
test_size = len(topics_list)
vec_env_test = vec_env

#### TAR Eval

In [ ]:

df = pd.DataFrame()
df_all_runs = pd.DataFrame()

for run in range(total_runs):

  # Test the trained agent
  vec_env_test = vec_env
  obs = vec_env_test.reset()
  test_steps = 100


  n_env = test_size
  agent=0
  target=0
  agent_vector=[]
  terminal_observation=[]

  topics = []
  recalls = []
  costs=[]
  e_costs = []
  reliabilities = []
  rewards = []
  distances = []
  differences = []
  targets = []
  run_cnts = []

  for eID in range(test_size):

    env = vec_env_test.envs[eID]
    obs, info = env.reset()

    for step in range(test_steps):
      action, _ = model.predict(obs, deterministic=False) # predict all next steps
      obs, reward, done, trun,info = env.step(action)


      if done or trun:
                  topic_id = info['topic_id']
                  recall = info['recall']
                  cost = info['cost']
                  e_cost =  ((info['agent'] - info['target']) / (100-info['target']))
                  distance = info['distance']

                  agent = info['agent']
                  target = info['target']
                  agent_vector = info['agent_vector']
                  terminal_observation = info['terminal_observation']

                  difference = target_recall - recall

                  reliability = 1 if recall >= target_recall else 0
                  topics.append(topic_id)
                  recalls.append(recall)
                  costs.append(cost)
                  e_costs.append(e_cost)
                  reliabilities.append(reliability)
                  rewards.append(reward)
                  distances.append(distance)
                  targets.append(target)
                  run_cnts.append(run)
                  differences.append(difference)

                  df_tmp = pd.DataFrame( list(zip([dataset_name]*len(topics_list), topics, run_cnts, recalls, reliabilities, costs, e_costs, rewards, differences, distances, targets)),
                  columns =['Dataset', 'Topic', 'Run', 'Recall', 'Reliability', 'Cost', 'e-Cost', 'Reward', 'Difference', 'Distance', 'Target'])

                  df = pd.concat([df_tmp])

                  df.groupby('Topic').mean()

                  break

  display(df)
  df.groupby('Topic').mean()
  df_all_runs = pd.concat([df_all_runs, df])

df_all_runs['Model'] = model_name
df_all_runs['Model_settings'] = tb_log_name
df_all_runs['Target_Recall'] = target_recall
df_all_runs

In [ ]:
display(df_all_runs.groupby('Topic').mean())
display(df_all_runs.groupby('Topic').std())


#### df_all_targets

In [ ]:

df_all_targets = pd.concat([df_all_targets, df_all_runs], ignore_index = True)


display(df_all_targets)

df_all_targets.describe()

In [ ]:
display(df_all_targets.groupby(['Target_Recall','Dataset']).mean().round(3))
display(df_all_targets.groupby(['Target_Recall','Dataset']).std().round(3))

###clef2019

In [ ]:

dataset_name = 'CLEF2019'


qrels = "data/qrels/CLEF2019_qrels_LiKs.txt" # use the same qrel list as their rankings


qrel_fname, query_rel_dic = load_rel_data(qrels)
print("Number of topics:", len(query_rel_dic))

run = "data/rankings/clef2019_ranking.txt"

doc_rank_dic, rank_rel_dic = load_run_data(run)

topics_list = make_topics_list(doc_rank_dic,1)  # sort topics by no docs

In [ ]:
#remove topic CD012164 last element, contains 61 items only, < 100 vector size
topics_list= topics_list[:-1]


In [ ]:
# Instantiate the vec env

#random topic selection for each env instance
SELECTED_TOPICS = [] # reset before/after each call, keep track of all randomly selected topics

vec_env = make_vec_env(TAREnv, n_envs=len(topics_list), env_kwargs=dict(target_recall=target_recall, topics_list = topics_list, topic_id=None, size=100, render_mode='human'))

SELECTED_TOPICS = [] # reset before/after each call, keep track of all randomly selected topics


In [ ]:
test_size = len(topics_list)
vec_env_test = vec_env

#### TAR Eval

In [ ]:

df = pd.DataFrame()
df_all_runs = pd.DataFrame()

for run in range(total_runs):

  # Test the trained agent
  vec_env_test = vec_env
  obs = vec_env_test.reset()
  test_steps = 100


  n_env = test_size
  agent=0
  target=0
  agent_vector=[]
  terminal_observation=[]

  topics = []
  recalls = []
  costs=[]
  e_costs = []
  reliabilities = []
  rewards = []
  distances = []
  differences = []
  targets = []
  run_cnts = []

  for eID in range(test_size):

    env = vec_env_test.envs[eID]
    obs, info = env.reset()

    for step in range(test_steps):
      action, _ = model.predict(obs, deterministic=False) # predict all next steps
      obs, reward, done, trun,info = env.step(action)


      if done or trun:
                  topic_id = info['topic_id']
                  recall = info['recall']
                  cost = info['cost']
                  e_cost =  ((info['agent'] - info['target']) / (100-info['target']))
                  distance = info['distance']

                  agent = info['agent']
                  target = info['target']
                  agent_vector = info['agent_vector']
                  terminal_observation = info['terminal_observation']

                  difference = target_recall - recall

                  reliability = 1 if recall >= target_recall else 0
                  topics.append(topic_id)
                  recalls.append(recall)
                  costs.append(cost)
                  e_costs.append(e_cost)
                  reliabilities.append(reliability)
                  rewards.append(reward)
                  distances.append(distance)
                  targets.append(target)
                  run_cnts.append(run)
                  differences.append(difference)

                  df_tmp = pd.DataFrame( list(zip([dataset_name]*len(topics_list), topics, run_cnts, recalls, reliabilities, costs, e_costs, rewards, differences, distances, targets)),
                  columns =['Dataset', 'Topic', 'Run', 'Recall', 'Reliability', 'Cost', 'e-Cost', 'Reward', 'Difference', 'Distance', 'Target'])

                  df = pd.concat([df_tmp])

                  df.groupby('Topic').mean()

                  break

  display(df)
  df.groupby('Topic').mean()
  df_all_runs = pd.concat([df_all_runs, df])

df_all_runs['Model'] = model_name
df_all_runs['Model_settings'] = tb_log_name
df_all_runs['Target_Recall'] = target_recall
df_all_runs

In [ ]:
display(df_all_runs.groupby('Topic').mean())
display(df_all_runs.groupby('Topic').std())


#### df_all_targets

In [ ]:

df_all_targets = pd.concat([df_all_targets, df_all_runs], ignore_index = True)


display(df_all_targets)

df_all_targets.describe()

In [ ]:
display(df_all_targets.groupby(['Target_Recall','Dataset']).mean().round(3))
display(df_all_targets.groupby(['Target_Recall','Dataset']).std().round(3))

##TREC-TR

### Training

In [ ]:
TRAINING = True


In [ ]:
training_dataset = 'TREC-TR'

#### sort topics by target location

In [ ]:


dataset_name = 'TREC-TR'

qrels = "data/qrels/TREC_TR_Training_qrels.txt"


qrel_fname, query_rel_dic = load_rel_data(qrels)
print("Number of topics:", len(query_rel_dic))

run = "data/rankings/tr_training_ranking.txt"

doc_rank_dic, rank_rel_dic = load_run_data(run)



topics_list = make_topics_list(doc_rank_dic,1)  # sort topics by no docs

In [ ]:
topics_info = []

for t in topics_list:
  topic_id, n_docs, n_rel, prev, target_location = load_topic_target_location(t,target_recall)
  print(topic_id, n_docs, n_rel, round(prev,3), target_location)
  topics_info.append([topic_id, n_docs, n_rel, prev, target_location])

topics_info

In [ ]:

df = pd.DataFrame(topics_info, columns=['topic_id', 'n_docs', 'n_rel', 'prev', 'target_location'])
df = df.sort_values(by=['target_location'])
df

In [ ]:
sorted_target_loc_topics = list(df['topic_id'])
sorted_target_loc_topics

####ordered topics

In [ ]:
TRAINING = True

SELECTED_TOPICS_ORDERERD = sorted_target_loc_topics
SELECTED_TOPICS_ORDERERD_INDEX = 0
# Instantiate the vec env

#random topic selection for each env instance
SELECTED_TOPICS = [] # reset before/after each call, keep track of all randomly selected topics

vec_env = make_vec_env(TAREnv, n_envs=len(topics_list), env_kwargs=dict(target_recall=target_recall, topics_list = topics_list, topic_id=None, size=100, render_mode='human'))

SELECTED_TOPICS = [] # reset before/after each call, keep track of all randomly selected topics

train_size = len(topics_list)
vec_env_train = vec_env

####  PPO

In [ ]:


tb_log_name = model_name+"_"+training_dataset+"_ppo_gma_"+str(gamma)+"_nsteps"+str(n_steps)+"_btch"+str(batch_size)+"_timesteps_"+str(total_timesteps)+ "_ent_coef"+str(ent_coef)+ learning_rate_type +"_n_epochs"+str(n_epochs)+"_target"+str(target_recall)

model = PPO(
    policy = 'MlpPolicy',
    env = vec_env_train,
    n_steps = n_steps,
    batch_size = batch_size,
    n_epochs = n_epochs,
    gamma = gamma,
    gae_lambda = 0.98,
    ent_coef = ent_coef,
    verbose=1,
    learning_rate = learning_rate,
    seed=0,
    tensorboard_log= tensorboard_log)

model.learn(total_timesteps=total_timesteps, tb_log_name=tb_log_name)


model.save(tensorboard_log+'model_'+tb_log_name)




In [ ]:
%tensorboard --logdir "$tensorboard_log"


### Testing

In [ ]:
dataset_name = 'TREC-TR'




qrels = "data/qrels/TREC_TR_Test_qrels.txt" # use the same qrel list as their rankings


qrel_fname, query_rel_dic = load_rel_data(qrels)
print("Number of topics:", len(query_rel_dic))

run = "data/rankings/tr_test_ranking.txt"

doc_rank_dic, rank_rel_dic = load_run_data(run)

topics_list = make_topics_list(doc_rank_dic,1)  # sort topics by no docs


In [ ]:
# Instantiate the vec env

#random topic selection for each env instance
SELECTED_TOPICS = [] # reset before/after each call, keep track of all randomly selected topics

vec_env = make_vec_env(TAREnv, n_envs=len(topics_list), env_kwargs=dict(target_recall=target_recall, topics_list = topics_list, topic_id=None, size=100, render_mode='human'))

SELECTED_TOPICS = [] # reset before/after each call, keep track of all randomly selected topics


In [ ]:
test_size = len(topics_list)
vec_env_test = vec_env

#### TAR Eval

In [ ]:

df = pd.DataFrame()
df_all_runs = pd.DataFrame()

for run in range(total_runs):

  # Test the trained agent
  vec_env_test = vec_env
  obs = vec_env_test.reset()
  test_steps = 100


  n_env = test_size
  agent=0
  target=0
  agent_vector=[]
  terminal_observation=[]

  topics = []
  recalls = []
  costs=[]
  e_costs = []
  reliabilities = []
  rewards = []
  distances = []
  differences = []
  targets = []
  run_cnts = []

  for eID in range(test_size):

    env = vec_env_test.envs[eID]
    obs, info = env.reset()

    for step in range(test_steps):
      action, _ = model.predict(obs, deterministic=False) # predict all next steps
      obs, reward, done, trun,info = env.step(action)


      if done or trun:
                  topic_id = info['topic_id']
                  recall = info['recall']
                  cost = info['cost']
                  e_cost =  ((info['agent'] - info['target']) / (100-info['target']))
                  distance = info['distance']

                  agent = info['agent']
                  target = info['target']
                  agent_vector = info['agent_vector']
                  terminal_observation = info['terminal_observation']

                  difference = target_recall - recall

                  reliability = 1 if recall >= target_recall else 0
                  topics.append(topic_id)
                  recalls.append(recall)
                  costs.append(cost)
                  e_costs.append(e_cost)
                  reliabilities.append(reliability)
                  rewards.append(reward)
                  distances.append(distance)
                  targets.append(target)
                  run_cnts.append(run)
                  differences.append(difference)

                  df_tmp = pd.DataFrame( list(zip([dataset_name]*len(topics_list), topics, run_cnts, recalls, reliabilities, costs, e_costs, rewards, differences, distances, targets)),
                  columns =['Dataset', 'Topic', 'Run', 'Recall', 'Reliability', 'Cost', 'e-Cost', 'Reward', 'Difference', 'Distance', 'Target'])

                  df = pd.concat([df_tmp])

                  df.groupby('Topic').mean()

                  break

  display(df)
  df.groupby('Topic').mean()
  df_all_runs = pd.concat([df_all_runs, df])

df_all_runs['Model'] = model_name
df_all_runs['Model_settings'] = tb_log_name
df_all_runs['Target_Recall'] = target_recall
df_all_runs

In [ ]:
display(df_all_runs.groupby('Topic').mean())
display(df_all_runs.groupby('Topic').std())


#### df_all_targets

In [ ]:

df_all_targets = pd.concat([df_all_targets, df_all_runs], ignore_index = True)


display(df_all_targets)

df_all_targets.describe()

In [ ]:
display(df_all_targets.groupby(['Target_Recall','Dataset']).mean().round(3))
display(df_all_targets.groupby(['Target_Recall','Dataset']).std().round(3))

##RCV1

### Training

In [ ]:
TRAINING = True


In [ ]:
training_dataset = 'RCV1'

#### sort topics by target location

In [ ]:
# LOAD RUN DATA
def load_run_data(run):
  run_fname = os.path.join(DIR, run)
  with open(run_fname, 'r', encoding='utf-8-sig') as infile: # resolve file encoding problem !!
    run_data = infile.readlines()
  doc_rank_dic = make_rank_dic(run_data)  # make dictionary of ranked docids for each queryid
  rank_rel_dic = make_rank_rel_dic(query_rel_dic,doc_rank_dic) # make dic of list relevances of ranked docs for each queryid

  #return doc_rank_dic, rank_rel_dic, rank_text_dic
  return doc_rank_dic, rank_rel_dic




dataset_name = 'RCV1'

qrels = "data/qrels/rcv1_qrels_selected_wo45_0.2.txt" # use the same qrel list as their rankings


qrel_fname, query_rel_dic = load_rel_data(qrels)
print("Number of topics:", len(query_rel_dic))

run = "data/rankings/RCV1_test_20wo45_ranking.txt"
run = "data/rankings/temp/RCV1_test_20wo45_ranking_utf8_2.txt"

doc_rank_dic, rank_rel_dic = load_run_data(run)###



topics_list = make_topics_list(doc_rank_dic,1)  # sort topics by no docs

In [ ]:
# only selected 1st level topics
topics_list = ['C151', 'C171', 'C181', 'C311', 'C331', 'C411', 'E121', 'E131', 'E141', 'E211', 'E311', 'E411', 'E511', 'G151', 'M131', 'M141']

In [ ]:
topics_info = []

for t in topics_list:
  topic_id, n_docs, n_rel, prev, target_location = load_topic_target_location(t,target_recall)
  print(topic_id, n_docs, n_rel, round(prev,3), target_location)
  topics_info.append([topic_id, n_docs, n_rel, prev, target_location])

topics_info

In [ ]:
# import pandas as pd
import pandas as pd


df = pd.DataFrame(topics_info, columns=['topic_id', 'n_docs', 'n_rel', 'prev', 'target_location'])
df = df.sort_values(by=['target_location'])
#df = df.sort_values(by=['target_location'],ascending=False)
df

In [ ]:
sorted_target_loc_topics = list(df['topic_id'])
sorted_target_loc_topics

####ordered topics

In [ ]:
TRAINING = True

SELECTED_TOPICS_ORDERERD = sorted_target_loc_topics
SELECTED_TOPICS_ORDERERD_INDEX = 0
# Instantiate the vec env

#random topic selection for each env instance
SELECTED_TOPICS = [] # reset before/after each call, keep track of all randomly selected topics

vec_env = make_vec_env(TAREnv, n_envs=len(topics_list), env_kwargs=dict(target_recall=target_recall, topics_list = topics_list, topic_id=None, size=100, render_mode='human'))

SELECTED_TOPICS = [] # reset before/after each call, keep track of all randomly selected topics

train_size = len(topics_list)
vec_env_train = vec_env

#### PPO

In [ ]:


tb_log_name = model_name+"_"+training_dataset+"_ppo_gma_"+str(gamma)+"_nsteps"+str(n_steps)+"_btch"+str(batch_size)+"_timesteps_"+str(total_timesteps)+ "_ent_coef"+str(ent_coef)+ learning_rate_type +"_n_epochs"+str(n_epochs)+"_target"+str(target_recall)

model = PPO(
    policy = 'MlpPolicy',
    env = vec_env_train,
    n_steps = n_steps,
    batch_size = batch_size,
    n_epochs = n_epochs,
    gamma = gamma,
    gae_lambda = 0.98,
    ent_coef = ent_coef,
    verbose=1,
    learning_rate = learning_rate,
    seed=0,
    tensorboard_log= tensorboard_log)

model.learn(total_timesteps=total_timesteps, tb_log_name=tb_log_name)


model.save(tensorboard_log+'model_'+tb_log_name)




In [ ]:
%tensorboard --logdir "$tensorboard_log"


### Testing

In [ ]:
dataset_name = 'RCV1'


qrels = "data/qrels/rcv1_qrels_selected_45_0.2.txt" # use the same qrel list as their rankings


qrel_fname, query_rel_dic = load_rel_data(qrels)
print("Number of topics:", len(query_rel_dic))

run = "data/rankings/RCV1_selected_45_0.2_ranking.txt"

doc_rank_dic, rank_rel_dic = load_run_data(run)

topics_list = make_topics_list(doc_rank_dic,1)  # sort topics by no docs


In [ ]:
# Instantiate the vec env

#random topic selection for each env instance
SELECTED_TOPICS = [] # reset before/after each call, keep track of all randomly selected topics

vec_env = make_vec_env(TAREnv, n_envs=len(topics_list), env_kwargs=dict(target_recall=target_recall, topics_list = topics_list, topic_id=None, size=100, render_mode='human'))

SELECTED_TOPICS = [] # reset before/after each call, keep track of all randomly selected topics


In [ ]:
test_size = len(topics_list)
vec_env_test = vec_env

#### TAR Eval

In [ ]:

df = pd.DataFrame()
df_all_runs = pd.DataFrame()

for run in range(total_runs):

  # Test the trained agent
  vec_env_test = vec_env
  obs = vec_env_test.reset()
  test_steps = 100


  n_env = test_size
  agent=0
  target=0
  agent_vector=[]
  terminal_observation=[]

  topics = []
  recalls = []
  costs=[]
  e_costs = []
  reliabilities = []
  rewards = []
  distances = []
  differences = []
  targets = []
  run_cnts = []

  for eID in range(test_size):

    env = vec_env_test.envs[eID]
    obs, info = env.reset()

    for step in range(test_steps):
      action, _ = model.predict(obs, deterministic=False) # predict all next steps
      obs, reward, done, trun,info = env.step(action)


      if done or trun:
                  topic_id = info['topic_id']
                  recall = info['recall']
                  cost = info['cost']
                  e_cost =  ((info['agent'] - info['target']) / (100-info['target']))
                  distance = info['distance']

                  agent = info['agent']
                  target = info['target']
                  agent_vector = info['agent_vector']
                  terminal_observation = info['terminal_observation']

                  difference = target_recall - recall

                  reliability = 1 if recall >= target_recall else 0
                  topics.append(topic_id)
                  recalls.append(recall)
                  costs.append(cost)
                  e_costs.append(e_cost)
                  reliabilities.append(reliability)
                  rewards.append(reward)
                  distances.append(distance)
                  targets.append(target)
                  run_cnts.append(run)
                  differences.append(difference)

                  df_tmp = pd.DataFrame( list(zip([dataset_name]*len(topics_list), topics, run_cnts, recalls, reliabilities, costs, e_costs, rewards, differences, distances, targets)),
                  columns =['Dataset', 'Topic', 'Run', 'Recall', 'Reliability', 'Cost', 'e-Cost', 'Reward', 'Difference', 'Distance', 'Target'])

                  df = pd.concat([df_tmp])

                  df.groupby('Topic').mean()

                  break

  display(df)
  df.groupby('Topic').mean()
  df_all_runs = pd.concat([df_all_runs, df])

df_all_runs['Model'] = model_name
df_all_runs['Model_settings'] = tb_log_name
df_all_runs['Target_Recall'] = target_recall
df_all_runs

In [ ]:
display(df_all_runs.groupby('Topic').mean())
display(df_all_runs.groupby('Topic').std())


#### df_all_targets

In [ ]:

df_all_targets = pd.concat([df_all_targets, df_all_runs], ignore_index = True)


display(df_all_targets)

df_all_targets.describe()

In [ ]:
display(df_all_targets.groupby(['Target_Recall','Dataset']).mean().round(3))
display(df_all_targets.groupby(['Target_Recall','Dataset']).std().round(3))

### vis all datasets

In [ ]:
display(df_all_targets.groupby(['Target_Recall','Dataset']).mean().round(3))

In [ ]:
(df_all_targets[['Target_Recall', 'Dataset', 'Recall' , 'Reliability', 'Cost', 'e-Cost']].groupby(['Target_Recall','Dataset']).mean().round(3)).to_latex()


